In [22]:
import calc_footprint_FFP_climatology as myfootprint_s
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
import pyproj as proj
import cartopy.crs as ccrs
import traceback
import os
import glob
import footprint_funcs as ff
from rasterio.warp import calculate_default_transform, reproject, Resampling

### Read in data

In [23]:
#Name of data files in 'data' directory
fluxes = os.path.join('data','V91_105.dat')
sigv = os.path.join('data','VX91_105.dat')

def date_parse_sigv(doy,hr):
    '''
    Sigv date parser (pd.read_csv) for sigv files
    Change yr to year being processed 
    '''
    
    yr='2018'
    
    if '2400' in hr:
        hr = '000'
        return pd.datetime.strptime(f'{yr}{int(doy)+1}{int(hr):04}', '%Y%j%H%M')
    else:
        return pd.datetime.strptime(f'{yr}{doy}{int(hr):04}', '%Y%j%H%M')

#Read in data
flux_df = pd.read_csv(fluxes,delim_whitespace=True,parse_dates={'TIMESTAMP':[0,1,2]},date_parser=ff.date_parse,
                      index_col=0,na_values=['*****'])

sigv_df = pd.read_csv(sigv,delim_whitespace=True,parse_dates={'TIMESTAMP':[0,1]}, date_parser=date_parse_sigv,
                           index_col=0)

full_df = flux_df.merge(sigv_df,left_index=True, right_index=True)

#Mask out certain wind directions
#Put bad wind directions in .isbetween() call
full_df = full_df[~full_df['Dir'].between(330.,360.) &
                  ~full_df['Dir'].between(75.,115.)]

### Calculate UTM from lat/lon for station to be used as an origin point in the model

In [24]:
#Name of station (choose from ['vernal', 'bloomfield', 'big_piney', 'palisade'])
station = 'vernal'

#Station name : [lat/lon (wgs84), utm_zone_epsg]
station_dict = {'vernal' : [(-109.5626, 40.45829), 'EPSG:32612'],
                'bloomfield' : [(-107.913830, 36.690893), 'EPSG:32613'],
                'big_piney' : [(-110.193893, 42.540674), 'EPSG:32612'],
                'palisade' : [(-108.3703720, 39.0935410), 'EPSG:32612']
               }

#Convert station lat/lon to local UTM
station_coord = station_dict[station][0]
in_proj = proj.Proj(init='EPSG:4326')
out_proj = proj.Proj(init=station_dict[station][1])
(station_x,station_y) = proj.transform(in_proj,out_proj,*station_coord)

### Run the model and save outputs in UTM

In [25]:
#Model parameters
zm_s = 2.78 #Measurement height [m]
h_c = 0.05 #Height of canopy [m]
h_s = 2000. #Height of boundary layer [m]
full_df['ol'] = zm_s/full_df['z/L'] #Monin-Obukhov length [m]
dx = 3. #Model resolution [m]
origin_d = 600. #Model bounds distance from origin [m]
start_hr = 7
end_hr = 20
hours = np.arange(start_hr,end_hr+1) #np.ndarray of hours

#Loop through each day in the dataframe
for doy in full_df.index.dayofyear.drop_duplicates().values:
    
    #Subset dataframe to only values in day of year
    print(f'Day of year: {doy}')
    temp_df = full_df[full_df.index.dayofyear == doy]
    new_dat = None

    for indx,t in enumerate(hours):
        
        band = indx + 1
        print(f'Hour: {t}')

        try:

            temp_line = temp_df.loc[temp_df.index.hour == t,:]
            
            #Calculate footprint
            temp_ffp = myfootprint_s.FFP_climatology(domain=[-origin_d,origin_d,-origin_d,origin_d],dx=dx,dy=dx,
                                    zm=zm_s,z0=h_c*.15,h=h_s,rs=None,
                                    ol=temp_line['ol'].values,sigmav=temp_line['SDVn'].values,
                                    umean=temp_line['V'].values, ustar=temp_line['ustar'].values,
                                    wind_dir=temp_line['Dir'].values,
                                    crop=0,fig=0,verbosity=0)

            f_2d = np.array(temp_ffp['fclim_2d'])
            x_2d = np.array(temp_ffp['x_2d']) + station_x
            y_2d = np.array(temp_ffp['y_2d']) + station_y
            f_2d = f_2d*dx**2

            #Calculate affine transform for given x_2d and y_2d
            affine_transform = ff.find_transform(x_2d,y_2d)
            
            #Create data file if not already created
            if new_dat is None:
                out_f = os.path.join('output','utm_footprints',f'{doy}_{station}.tif')
                print(f_2d.shape)
                new_dat = rasterio.open(out_f,'w',driver='GTiff',dtype=rasterio.float64,
                                        count=len(hours),height=f_2d.shape[0],width=f_2d.shape[1],
                                        transform=affine_transform,
                                        crs=out_proj.srs,
                                        nodata=0.00000000e+000)

        except Exception as e:

            print(f'Hour {t} footprint failed, band {band} not written.')

            temp_ffp = None

            continue

        #Mask out points that are below a % threshold (defaults to 90%)
        f_2d = ff.mask_fp_cutoff(f_2d)

        #Write the new band
        new_dat.write(f_2d,indx+1)

        #Update tags with metadata
        tag_dict = {'hour':f'{t*100:04}',
                    'wind_dir':temp_line['Dir'].values,
                    'total_footprint':np.nansum(f_2d)}

        new_dat.update_tags(indx+1,**tag_dict)
    
    #Close dataset if it exists
    try:
        new_dat.close()
    except:
        continue
        
    print()
    

Day of year: 91
Hour: 7
(401, 401)
Hour: 8
Hour: 9
operands could not be broadcast together with shapes (401,401) (0,) 
Hour 9 footprint failed, band 3 not written.
Hour: 10
operands could not be broadcast together with shapes (401,401) (0,) 
Hour 10 footprint failed, band 4 not written.
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20

Day of year: 92
Hour: 7

Hour 7 footprint failed, band 1 not written.
Hour: 8

Hour 8 footprint failed, band 2 not written.
Hour: 9
(401, 401)
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20

Day of year: 93
Hour: 7
(401, 401)
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20

Hour 20 footprint failed, band 14 not written.

Day of year: 94
Hour: 7

Hour 7 footprint failed, band 1 not written.
Hour: 8
(401, 401)
Hour: 9

Hour 9 footprint failed, band 3 not written.
Hour: 10
operands could not be broa

Hour: 17
Hour: 18
Hour: 19
operands could not be broadcast together with shapes (401,401) (0,) 
Hour 19 footprint failed, band 13 not written.
Hour: 20

Day of year: 105
Hour: 7
(401, 401)
Hour: 8
operands could not be broadcast together with shapes (401,401) (0,) 
Hour 8 footprint failed, band 2 not written.
Hour: 9
Hour: 10
operands could not be broadcast together with shapes (401,401) (0,) 
Hour 10 footprint failed, band 4 not written.
Hour: 11
operands could not be broadcast together with shapes (401,401) (0,) 
Hour 11 footprint failed, band 5 not written.
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18

Hour 18 footprint failed, band 12 not written.
Hour: 19
Hour: 20

Hour 20 footprint failed, band 14 not written.



### Reproject from UTM to custom projection

In [26]:
#Get file paths and names
in_file_paths = glob.glob(os.path.join('output','utm_footprints','*.tif'))
files = [os.path.basename(f) for f in in_file_paths]
out_file_paths = [os.path.join('output','custom_footprints',f) for f in files]

#Get crs from sample image
test_img = 'p37r32_20170629_vernaleddysite_reflsurf_l8.tif'
test_raster = rasterio.open(test_img)
dst_crs = test_raster.crs
test_raster.close()

#Reproject image
for in_f, out_f in zip(in_file_paths, out_file_paths):
    
    #Read in old file
    with rasterio.open(in_f,'r+') as src:
        transform,width,height = calculate_default_transform(src.crs,dst_crs,src.width,src.height,*src.bounds,resolution=3.)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        
        #Reproject old file and write to new file
        with rasterio.open(out_f,'w',**kwargs) as dst:
            for i in range(1, src.count+1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs)
                try:
                    dst.update_tags(i,**src.tags(i))
                except:
                    continue